# Intro to HDF5 data model

High-level overview of the HDF5 file structure and basic tools

![hdf5-wheel](images/hdf5-wheel.png)  
Source: © Copyright The HDF Group  

## What is HDF5?

* HDF5 = Hierarchical Data Format Version 5
* A file format optimized for numeric data
* A hierarquichal structure to store information (like folders)
* A self-describing container: Metadata + Data
* A library with several functionalities (e.g. command line tools)
* It is high level from user side (easy access)
* It is low level from machine side (binary, compressible)
* Fast I/O, parallel read/write (!), very good for HPC
* Data can be read/written in chuncks, in-memory, out-of-memory  

Read more: [https://www.hdfgroup.org/solutions/hdf5/](https://www.hdfgroup.org/solutions/hdf5/) 

## How popular is it?

* Matlab `*.m` files **are** HDF5!
* NetCDF4 files **are** HDF5!
* Keras/TensorFlow saves ML model weights to HDF5
* ICESat-2 data comes in HDF5

## Write data to HDF5

Let's create some fake data

In [ ]:
%matplotlib inline

In [ ]:
import h5py
import numpy as np

x = np.random.randn(100)
y = np.random.randn(100)
z = np.random.randn(100)

print(x)

Save 1D arrays to file

In [ ]:
with h5py.File('myfile.h5', 'w') as f:  # open file in write mode
    f['x'] = x                          # write data
    f['y'] = y
    f['z'] = z

In [ ]:
!ls *.h5  # Check the file was created

The HDF5 library comes with some useful command-line tools  
There is no need to write code to inspect an HDF5 file!

In [ ]:
!h5ls myfile.h5  # inspect the file w/command-line tools

> **NOTE:**  
> We wil see more sophysticated command-line tools below.


## Read data from HDF5

Load data (in memory) vs. get pointer (out of memory)

In [ ]:
with h5py.File('myfile.h5', 'r') as f:  # open file
    x = f['x'][:]                       # read data into memory
    y = f['y']                          # get pointer to data on disk
    
    print('x (in mem): ', x)
    print('y (on disk):', y)
    print('')
    print('x (in mem): ', type(x))
    print('y (on disk):', type(y))
    print('')
    print('x (in mem): ', x.shape)
    print('y (on disk):', y.shape)  # same info from out-of-memory array

## Append data to HDF5

Let's add some data with specific paths (groups)

In [ ]:
with h5py.File('myfile.h5', 'a') as f:
    f['/path/to/data/vec'] = z**2
    f['/path/to/data/mat'] = z.reshape(10,10)

> **NOTE:**  
> `path`, `to` and `data` are groups  
> `vec` and `mat` are datasets  

In [ ]:
# Inspect file from the command line
!h5ls -r myfile.h5

Now our HDF5 file has some structure!

## Add metadata to HDF5

Let's first inpect the metadata that's added by default

In [ ]:
# Inspect Metadate from the commaand line
!h5dump -H myfile.h5

Let's add our own metadata

In [ ]:
with h5py.File('myfile.h5', 'a') as f:
    g = f['/path']               # pointer to group 'path'
    d = f['/path/to/data/mat']   # pointer to dataset 'mat'
    
    # Metadata for the group
    g.attrs['Description'] = 'This is a group'
    g.attrs['Author'] = 'Your name'
    g.attrs['email'] = 'yourname@domain.com'
    
    # Metadata for the data
    d.attrs['Description'] = 'This is an array'
    d.attrs['Date'] = '2020-07-01'
    d.attrs['Version'] = '1.2'

In [ ]:
# Inspect Metadata from the commaand line
!h5dump --header myfile.h5

## Copy some data to another HDF5

In [ ]:
%%bash

h5copy -i myfile.h5 -o myfile2.h5 -s x -d x

h5copy -i myfile.h5 -o myfile2.h5 -s /path/to/data/mat -d mat

ls *.h5
h5ls -r myfile2.h5

Let's check the differences between original and copy files 

In [ ]:
!h5diff -v myfile.h5 myfile2.h5

## Inspect HDF5 from Python

Let's do the same as above but using Python code

In [ ]:
f = h5py.File('myfile.h5', 'r')  # keep it open

print(f.keys())  # Inspect base groups quickly

Inspect the full structure w/metadata

In [ ]:
def print_attrs(name, obj):
    print(name)
    for key, val in obj.attrs.items():
        print("    %s: %s" % (key, val))

f.visititems(print_attrs)        

In [ ]:
f.close()

## Create an extendable dataset

Create an empty container (called `grids`) extendable in the 3rd dim

In [ ]:
with h5py.File('myfile.h5', 'a') as f:
    dset = f.create_dataset(
        "grids", 
        (10,10,5), 
        maxshape=(10,10,None), 
        dtype='f4', 
        chunks=True,
        compression='gzip')

Check that our created container has an infinity dimension

In [ ]:
!h5ls -r myfile.h5

Create a bunch of 2D grids to save to our empty container

In [ ]:
mygrids = [np.random.randn(10,10) for _ in range(5)]

print(np.shape(mygrids))  # 5 grids of 10 by 10

Save grids one at a time and close the file

In [ ]:
with h5py.File('myfile.h5', 'a') as f:
    grids = f['grids']
    
    for k,g in enumerate(mygrids):
        grids[:,:,k] = g

Read in (select) specific grids with fancy indexing

In [ ]:
with h5py.File('myfile.h5', 'r') as f:
    mygrids = f['grids'][:,:,[0,2,4]]  # 3 grids out of 5
    
print(np.shape(mygrids))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Plot each grid to check dimensions are right
[plt.matshow(mygrids[:,:,k]) for k in range(mygrids.shape[2])]
plt.show()

## What about the cloud?

The HDF Group (the organization behind HDF5) offers a cloud solution for HDF5:

- [Highly Scalable Data Service (HSDS)](https://www.hdfgroup.org/solutions/highly-scalable-data-service-hsds/) - an interface for reading and writing HDF5 within object-based storage environments such as the Cloud.

The HDF Group also offers commercial product services around HSDS:

- [Kita Lab](https://www.hdfgroup.org/hdfkitalab/) - a managed JupyterLab environment with a command line interface (for small groups)
- [Kita Server for AWS Marketplace](https://www.hdfgroup.org/solutions/highly-scalable-data-service-hsds/) - for those using ASW services

## The Zarr library (a newcomer)

![Zarr](images/zarr-logo.png)  
Source: © Copyright Zarr Developers

> Zarr stores each chunk of a dataset as a separate object in Cloud object storage,  
> making it efficient for clusters of CPUs to access the data in parallel.  
> It also allows all the metadata to be in a single location which requires just one read.

How does it compare to HDF5?

- The Zarr data model and library offer the same capabilities as HDF5 + _added functionality_
- Both `zarr` and `h5py` have an (almost) identical interface to access the data
- Zarr is written in Python, and Python oriented. HDF5 is written in C, and cross language
- Zarr is less than 5 years old (young project). HDF5 is over 20 years old (mature project)
- Zarr is being developed from the bottom-up as cloud performant. HDF5 is being adapted

What is the added functionality?

- Many more compression libraries out of the box
- Composible filters (e.g. you can combine different compression algorithms)
- More storage backends (e.g. cloud specific: S3 and Azure mappings)
- Easer to extend (e.g. you can write custom storage backends, in pure Python)
- Native support for multithreading (e.g. concurrent writes to the same file)

Checkout Joe Jevnik's talk about the differences between Zarr and HDF5:

* [Zarr vs. HDF5](https://www.youtube.com/watch?v=-l445lCPTts)

## Final thoughts on HDF5

- Many small files is usually more practical than a few large ones (for processing)
- Read/Write is faster on smaller files (faster queries)
- Network transfer is usually faster with a bunch of smaller files
- Storing a lot of data into a single file is susceptible to corruption
- Many small files simplifies (embarrasingly) parallelization

## Bonus: HDFView Software

If you like GUIs, [HDFView](https://www.hdfgroup.org/downloads/hdfview/) is a visual tool written in Java for browsing and editing HDF5 and HDF4 files. With HDFView you can:

- View a file hierarchy in a tree structure
- Create new files, add or delete groups and datasets
- View and modify the content of a dataset
- Add, delete and modify attributes

![hdfview](images/hdfview.png)